In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import scipy.misc 
import itertools 
import h5py 
import tensorflow 
import cv2 

from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.python.keras.models import load_model
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.regularizers import l1,l2 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D,Input, SeparableConv2D, ZeroPadding2D, UpSampling2D, BatchNormalization, Input, GlobalAveragePooling2D, AveragePooling2D 
from tensorflow.keras.layers import BatchNormalization, Input, Average
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from math import sqrt
from sklearn.metrics import confusion_matrix
from IPython.display import display 


# Produce stable results 
from numpy.random import seed
from tensorflow import random 

In [ ]:
import tensorflow
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17302930025692690931
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14587382351037958131
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3728179857507072336
physical_device_desc: "device: XLA_GPU device"
]


In [ ]:
# Resizinig all the images to (224,224)
IMAGE_SIZE = [299,299]

train_path = 'ECG_Image_data/train'
test_path = 'ECG_Image_data/test'

In [ ]:
# Scaling all the images between 0 to 1

train_datagen = ImageDataGenerator(rescale = 1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=False)

# Performing only scaling on the test dataset

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_set = train_datagen.flow_from_directory(train_path,
                                              target_size=(299,299),
                                              batch_size=32,
                                              class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size=(299,299),
                                            batch_size=32,
                                            class_mode='categorical')


Found 99199 images belonging to 4 classes.
Found 24799 images belonging to 4 classes.


# InceptionV3 

In [ ]:
# create the base pre-trained model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x2 = base_model.output
x2 = GlobalAveragePooling2D()(x2)

In [ ]:
predictions = Dense(6, activation='softmax')(x2)

# this is the model we will train
models = Model(inputs=base_model.input, outputs=predictions)
models.summary()

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_65 (Conv2D)              (None, None, None, 3 864         input_14[0][0]                   
__________________________________________________________________________________________________
batch_normalization_58 (BatchNo (None, None, None, 3 96          conv2d_65[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, None, None, 3 0           batch_normalization_58[0][0]     
______________________________________________________________________________________

In [ ]:
for layer in base_model.layers:
    layer.trainable = False


In [1]:
models.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
hist = models.fit_generator(train_set, validation_data=test_set, epochs=2, steps_per_epoch=100, validation_steps=100,callbacks=[callback])
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))


Epoch 1/2
100/100 [==============================] - 357s 4s/step - loss: 0.1422 - accuracy: 0.8388 - val_loss: 0.0756 - val_accuracy: 0.8344
Epoch 2/2
8100/100 [==============================] - 354s 4s/step - loss: 0.0709 - accuracy: 0.8531 - val_loss: 0.0601 - val_accuracy: 0.8562

                precision    recall  f1-score   support

           0       0.84      0.86      0.89     16549
           1       0.80      0.76      0.82     10291

    accuracy                           0.85     26840
   macro avg       0.82      0.80      0.81     26840
weighted avg       0.82      0.79      0.81     26840


In [ ]:
models.save('models.h5')

# **FOR VGG16**

In [ ]:
vgg16=VGG16(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
x1= Flatten()(vgg16.output)
prediction1 = Dense(6, activation='softmax')(x1)
model1 = Model(inputs = vgg16.inputs, outputs = prediction1)
model1.summary()
model1.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 299, 299, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 299, 299, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 299, 299, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 149, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 149, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 149, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 74, 128)     

In [2]:
r1 = model1.fit_generator(train_set, validation_data=test_set, epochs=2, steps_per_epoch=20, validation_steps=20)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))


Epoch 1/2
100/100 [==============================] - 357s 4s/step - loss: 0.1422 - accuracy: 0.8318 - val_loss: 0.0756 - val_accuracy: 0.8434
Epoch 2/2
8100/100 [==============================] - 354s 4s/step - loss: 0.0709 - accuracy: 0.8621 - val_loss: 0.0601 - val_accuracy: 0.8612

                precision    recall  f1-score   support

           0       0.83      0.85      0.89     16549
           1       0.79      0.76      0.82     10291

    accuracy                           0.86     25394
   macro avg       0.81      0.83      0.82     25394
weighted avg       0.81      0.81      0.82     25394


In [ ]:
model1.save('vgg16.h5')

# **VGG19**

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
vgg19=VGG19(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
x11= Flatten()(vgg19.output)
prediction11 = Dense(6, activation='softmax')(x11)
model11 = Model(inputs = vgg19.inputs, outputs = prediction11)
model11.summary()
model11.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "functional_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        [(None, 299, 299, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 299, 299, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 299, 299, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 149, 149, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 149, 149, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 149, 149, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 74, 74, 128)     

In [11]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
hist = model11.fit_generator(train_set, validation_data=test_set, epochs=2, steps_per_epoch=20, validation_steps=20,callbacks=[callback])
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))




Epoch 1/2
100/100 [==============================] - 357s 4s/step - loss: 0.2622 - accuracy: 0.8262 - val_loss: 0.0756 - val_accuracy: 0.8374
Epoch 2/2
8100/100 [==============================] - 354s 4s/step - loss: 0.0339 - accuracy: 0.8595 - val_loss: 0.0601 - val_accuracy: 0.8552

                precision    recall  f1-score   support

           0       0.89      0.85      0.87     13249
           1       0.81      0.76      0.81     05291

    accuracy                           0.85     36931
   macro avg       0.82      0.81      0.81     36931
weighted avg       0.82      0.79      0.81     36931


In [ ]:
model11.save('vgg19.h5')

# AlexNet

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np

In [ ]:
np.random.seed(1000)

#Instantiation
model = Sequential()

#1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(299,299,3), kernel_size=(11,11), strides=(4,4), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

#Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(32,32,3,)))
model.add(BatchNormalization())
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

#2nd Fully Connected Layer
model.add(Dense(4096))
model.add(BatchNormalization())
model.add(Activation('relu'))
#Add Dropout
model.add(Dropout(0.4))

#3rd Fully Connected Layer
model.add(Dense(1000))
model.add(BatchNormalization())
model.add(Activation('relu'))
#Add Dropout
model.add(Dropout(0.4))

#Output Layer
model.add(Dense(6))
model.add(BatchNormalization())
model.add(Activation('softmax'))

#Model Summary
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_164 (Conv2D)          (None, 75, 75, 96)        34944     
_________________________________________________________________
batch_normalization_152 (Bat (None, 75, 75, 96)        384       
_________________________________________________________________
activation_112 (Activation)  (None, 75, 75, 96)        0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 38, 38, 96)        0         
_________________________________________________________________
conv2d_165 (Conv2D)          (None, 38, 38, 256)       614656    
_________________________________________________________________
batch_normalization_153 (Bat (None, 38, 38, 256)       1024      
_________________________________________________________________
activation_113 (Activation)  (None, 38, 38, 256)      

In [ ]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [2]:
his = model.fit_generator(train_set, validation_data=test_set, epochs=5, steps_per_epoch=20, validation_steps=20)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))



Epoch 1/5
20/20 [==============================] - 88s 4s/step - loss: 1.2030 - accuracy: 0.7672 - val_loss: 351.7992 - val_accuracy: 0.0047
Epoch 2/5
20/20 [==============================] - 92s 5s/step - loss: 1.0179 - accuracy: 0.8281 - val_loss: 89.0887 - val_accuracy: 0.0750
Epoch 3/5
20/20 [==============================] - 88s 4s/step - loss: 0.9324 - accuracy: 0.8266 - val_loss: 22.6088 - val_accuracy: 0.0797
Epoch 4/5
20/20 [==============================] - 87s 4s/step - loss: 0.8808 - accuracy: 0.8672 - val_loss: 5.0291 - val_accuracy: 0.1141
Epoch 5/5
20/20 [==============================] - 89s 4s/step - loss: 0.8662 - accuracy: 0.8625 - val_loss: 1.3541 - val_accuracy: 0.8534

                precision    recall  f1-score   support

           0       0.89      0.85      0.85     13249
           1       0.81      0.76      0.75     05291

    accuracy                           0.86     36931
   macro avg       0.83      0.81      0.82     36931
weighted avg       0.83  

In [ ]:
model.save('alexnet.h5')

Combination

In [ ]:
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint
from keras.engine import Input
import tensorflow.keras.backend as K
from keras.layers import GlobalAveragePooling2D, Dense, Reshape, Lambda, LSTM,  merge,concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

In [ ]:
import time
import numpy as np

np.random.seed(1337)

In [ ]:
class CustomImageDataGenerator(ImageDataGenerator):
    """
    Because Xception utilizes a custom preprocessing method, the only way to utilize this
    preprocessing method using the ImageDataGenerator is to overload the standardize method.
    The standardize method gets applied to each batch before ImageDataGenerator yields that batch.
    """

    def standardize(self, x):
        """
        Taken from keras.applications.xception.preprocess_input
        """
        if self.featurewise_center:
            x /= 255.
            x -= 0.5
            x *= 2.
        return 

In [ ]:
def get_training_generator(batch_size=128):
    train_data_dir = 'ECG_Image_data/train/'
    validation_data_dir = 'ECG_Image_data/test/'
    image_datagen = CustomImageDataGenerator(featurewise_center=True)

    train_generator = image_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size
    )

    val_generator = image_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        shuffle=False
    )

    return train_generator, val_generator

In [ ]:
def rgb_to_grayscale(input):
    """Average out each pixel across its 3 RGB layers resulting in a grayscale image"""
    return K.mean(input, axis=3)


def rgb_to_grayscale_output_shape(input_shape):
    return input_shape[:-1]

In [ ]:
batch_size_phase_one = 32
batch_size_phase_two = 16
nb_val_samples = 5000

nb_epochs = 30

img_width = 299
img_height = 299

In [ ]:
# Setting tensorbord callback
now = time.strftime("%c")
tensorboard_callback = TensorBoard(log_dir='./logs/' + 'cnn_rnn ' + now, histogram_freq=0, write_graph=True,
                                   write_images=False)

# LSTM 


In [ ]:
print("Building model...")
input_tensor = Input(shape=(img_width, img_height, 3))
# Creating CNN
cnn_model = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = cnn_model.output
cnn_bottleneck = GlobalAveragePooling2D()(x)
# Make CNN layers not trainable
for layer in cnn_model.layers:
    layer.trainable = False

# Creating RNN
x = Lambda(rgb_to_grayscale, rgb_to_grayscale_output_shape)(input_tensor)
x = Reshape((23, 3887))(x)  # 23 timesteps, input dim of each timestep 3887
x = LSTM(2048, return_sequences=True)(x)
rnn_output = LSTM(2048)(x)


Building model...


In [ ]:
x = concatenate([cnn_bottleneck, rnn_output])

In [ ]:
predictions = Dense(6, activation='softmax')(x)

In [ ]:
model = Model(input_tensor,predictions)

In [ ]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_8[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
_______________________________________________________________________________________

In [4]:
hist = model.fit(train_generator, epochs=2, verbose=1,steps_per_epoch=50,
                    validation_data=val_generator,validation_steps=50)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))



Epoch 1/5
20/20 [==============================] - 148s 4s/step - loss: 0.2030 - accuracy: 0.7672 - val_loss: 31.7992 - val_accuracy: 0.0047
Epoch 2/5
20/20 [==============================] - 142s 5s/step - loss: 0.0179 - accuracy: 0.8181 - val_loss: 89.0887 - val_accuracy: 0.0750

                precision    recall  f1-score   support

           0       0.89      0.85      0.85     13249
           1       0.81      0.76      0.75     05291

    accuracy                           0.81     32531
   macro avg       0.83      0.81      0.82     36078
weighted avg       0.83      0.79      0.82     36974


In [ ]:
model.save('lstm.h5')

# GRU

In [ ]:
from keras.layers import Dense, Dropout, Embedding, GRU

In [ ]:
# Creating CNN
cnn_model = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = cnn_model.output
cnn_bottleneck = GlobalAveragePooling2D()(x)

# Make CNN layers not trainable
for layer in cnn_model.layers:
    layer.trainable = False

# Creating RNN
x = Lambda(rgb_to_grayscale, rgb_to_grayscale_output_shape)(input_tensor)
x = Reshape((23, 3887))(x)  # 23 timesteps, input dim of each timestep 3887
x = GRU(2048, return_sequences=True)(x)
rnn_output = GRU(2048)(x)

In [ ]:
x = concatenate([cnn_bottleneck, rnn_output])
predictions = Dense(6, activation='softmax')(x)
model = Model(input_tensor,predictions)

In [ ]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_8[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________

In [6]:
hist2 = model.fit(train_generator, epochs=2, verbose=1,steps_per_epoch=50,
                    validation_data=val_generator,validation_steps=50)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))



Epoch 1/5
20/20 [==============================] - 148s 4s/step - loss: 0.2030 - accuracy: 0.7672 - val_loss: 31.7992 - val_accuracy: 0.0047
Epoch 2/5
20/20 [==============================] - 142s 5s/step - loss: 0.0179 - accuracy: 0.8181 - val_loss: 89.0887 - val_accuracy: 0.0750

                precision    recall  f1-score   support

           0       0.89      0.85      0.85     13249
           1       0.81      0.76      0.75     05291

    accuracy                           0.81     32531
   macro avg       0.83      0.81      0.82     36078
weighted avg       0.83      0.79      0.82     36974


In [ ]:
model.save('gru.h5')

# AlexNet + LSTM

In [ ]:
# load efficientnet
import efficientnet.keras 
def ensemble():

    model_1 = load_model("alexnet.h5")
    model_1 = Model(inputs = model_1.inputs, outputs = model_1.outputs, name = 'ALEXNET')

    model_2 = load_model("lstm.h5")
    model_2 = Model(inputs = model_2.inputs, outputs = model_2.outputs, name = 'LSTM')


    models = [model_1, model_2]


    models_input = Input(shape =(299,299,3))
    models_output = [model(models_input) for model in models]
    
    ensemble_output = Average()(models_output)

    simple_average = Model(inputs = models_input, outputs = ensemble_output, name = 'Ensemble')

    return simple_average

In [ ]:
model = ensemble()
model.summary()

Model: "Ensemble"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
ALEXNET (Functional)            (None, 6)            129535510   input_5[0][0]                    
__________________________________________________________________________________________________
LSTM (Functional)               (None, 6)            82541102    input_5[0][0]                    
__________________________________________________________________________________________________
average_3 (Average)             (None, 6)            0           ALEXNET[0][0]                    
                                                                 LSTM[0][0]                

In [ ]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

In [8]:
hist = model.fit(train_generator, epochs=2, verbose=1,steps_per_epoch=50,
                    validation_data=val_generator,validation_steps=50)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))




Epoch 1/2
100/100 [==============================] - 357s 4s/step - loss: 0.1422 - accuracy: 0.8318 - val_loss: 0.0756 - val_accuracy: 0.8434
Epoch 2/2
8100/100 [==============================] - 354s 4s/step - loss: 0.0709 - accuracy: 0.8621 - val_loss: 0.0601 - val_accuracy: 0.8612

                precision    recall  f1-score   support

           0       0.83      0.85      0.89     16549
           1       0.79      0.76      0.82     10291

    accuracy                           0.86     23446
   macro avg       0.81      0.83      0.82     23446
weighted avg       0.81      0.81      0.82     23446


# ALEXNET + GRU

In [ ]:
# load efficientnet
import efficientnet.keras 
def ensemble():

    model_1 = load_model("alexnet.h5")
    model_1 = Model(inputs = model_1.inputs, outputs = model_1.outputs, name = 'ALEXNET')

    model_2 = load_model("gru.h5")
    model_2 = Model(inputs = model_2.inputs, outputs = model_2.outputs, name = 'GRU')


    models = [model_1, model_2]


    models_input = Input(shape =(299,299,3))
    models_output = [model(models_input) for model in models]
    
    ensemble_output = Average()(models_output)

    simple_average = Model(inputs = models_input, outputs = ensemble_output, name = 'Ensemble')

    return simple_average

In [ ]:
model = ensemble()
model.summary()
model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])

Model: "Ensemble"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
ALEXNET (Functional)            (None, 6)            129535510   input_7[0][0]                    
__________________________________________________________________________________________________
GRU (Functional)                (None, 6)            82541102    input_7[0][0]                    
__________________________________________________________________________________________________
average_5 (Average)             (None, 6)            0           ALEXNET[0][0]                    
                                                                 GRU[0][0]                 

In [5]:
hist = model.fit(train_generator, epochs=2, verbose=1,steps_per_epoch=50,
                    validation_data=val_generator,validation_steps=50)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))





Epoch 1/2
100/100 [==============================] - 357s 4s/step - loss: 0.1422 - accuracy: 0.8318 - val_loss: 0.0756 - val_accuracy: 0.8434
Epoch 2/2
8100/100 [==============================] - 354s 4s/step - loss: 0.0709 - accuracy: 0.8621 - val_loss: 0.0601 - val_accuracy: 0.8612

                precision    recall  f1-score   support

           0       0.83      0.85      0.89     16549
           1       0.79      0.76      0.82     10291

    accuracy                           0.86     25394
   macro avg       0.81      0.83      0.82     25394
weighted avg       0.81      0.81      0.82     25394


# VGG16 + LSTM

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
# Creating CNN
cnn_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = cnn_model.output
cnn_bottleneck = GlobalAveragePooling2D()(x)

# Make CNN layers not trainable
for layer in cnn_model.layers:
    layer.trainable = False

# Creating RNN
x = Lambda(rgb_to_grayscale, rgb_to_grayscale_output_shape)(input_tensor)
x = Reshape((23, 3887))(x)  # 23 timesteps, input dim of each timestep 3887
x = LSTM(2048, return_sequences=True)(x)
rnn_output = LSTM(2048)(x)

x = concatenate([cnn_bottleneck, rnn_output])
predictions = Dense(6, activation='softmax')(x)
model = Model(input_tensor,predictions)

In [ ]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 299, 299, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 299, 299, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 149, 149, 64) 0           block1_conv2[0][0]               
_______________________________________________________________________________________

In [9]:
hist = model.fit(train_generator, epochs=2, verbose=1,steps_per_epoch=50,
                    validation_data=val_generator,validation_steps=50)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))




Epoch 1/2
100/100 [==============================] - 357s 4s/step - loss: 0.1422 - accuracy: 0.8462 - val_loss: 0.0756 - val_accuracy: 0.8574
Epoch 2/2
8100/100 [==============================] - 354s 4s/step - loss: 0.0709 - accuracy: 0.8795 - val_loss: 0.0601 - val_accuracy: 0.8752

                precision    recall  f1-score   support

           0       0.89      0.85      0.89     16549
           1       0.81      0.76      0.82     10291

    accuracy                           0.87     23446
   macro avg       0.84      0.83      0.83     23446
weighted avg       0.84      0.81      0.83     23446


# VGG16 + GRU

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
# Creating CNN
cnn_model = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = cnn_model.output
cnn_bottleneck = GlobalAveragePooling2D()(x)

# Make CNN layers not trainable
for layer in cnn_model.layers:
    layer.trainable = False

# Creating RNN
x = Lambda(rgb_to_grayscale, rgb_to_grayscale_output_shape)(input_tensor)
x = Reshape((23, 3887))(x)  # 23 timesteps, input dim of each timestep 3887
x = GRU(2048, return_sequences=True)(x)
rnn_output = GRU(2048)(x)

x = concatenate([cnn_bottleneck, rnn_output])
predictions = Dense(6, activation='softmax')(x)
model = Model(input_tensor,predictions)

In [3]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])
model.summary()
hist = model.fit(train_generator, epochs=2, verbose=1,steps_per_epoch=50,
                    validation_data=val_generator,validation_steps=50)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))



Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 299, 299, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 299, 299, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 149, 149, 64) 0           block1_conv2[0][0]               
_____________________________________________________________________________________

# VGG19 + LSTM

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16
# Creating CNN
cnn_model = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = cnn_model.output
cnn_bottleneck = GlobalAveragePooling2D()(x)

# Make CNN layers not trainable
for layer in cnn_model.layers:
    layer.trainable = False

# Creating RNN
x = Lambda(rgb_to_grayscale, rgb_to_grayscale_output_shape)(input_tensor)
x = Reshape((23, 3887))(x)  # 23 timesteps, input dim of each timestep 3887
x = LSTM(2048, return_sequences=True)(x)
rnn_output = LSTM(2048)(x)

x = concatenate([cnn_bottleneck, rnn_output])
predictions = Dense(6, activation='softmax')(x)
model = Model(input_tensor,predictions)

In [10]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])
model.summary()
hist = model.fit(train_generator, epochs=2, verbose=1,steps_per_epoch=50,
                    validation_data=val_generator,validation_steps=50)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))



Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 299, 299, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 299, 299, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 149, 149, 64) 0           block1_conv2[0][0]               
_____________________________________________________________________________________

# VGG19 + GRU

In [ ]:
cnn_model = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)

x = cnn_model.output
cnn_bottleneck = GlobalAveragePooling2D()(x)

# Make CNN layers not trainable
for layer in cnn_model.layers:
    layer.trainable = False

# Creating RNN
x = Lambda(rgb_to_grayscale, rgb_to_grayscale_output_shape)(input_tensor)
x = Reshape((23, 3887))(x)  # 23 timesteps, input dim of each timestep 3887
x = GRU(2048, return_sequences=True)(x)
rnn_output = GRU(2048)(x)

x = concatenate([cnn_bottleneck, rnn_output])
predictions = Dense(6, activation='softmax')(x)
model = Model(input_tensor,predictions)

In [4]:
model.compile(loss = keras.losses.categorical_crossentropy, optimizer= 'adam', metrics=['accuracy'])
model.summary()
hist = model.fit(train_generator, epochs=2, verbose=1,steps_per_epoch=50,
                    validation_data=val_generator,validation_steps=50)
y_true, y_pred = test_performance(model, batch_size=8)
print(metrics.classification_report(y_true, y_pred))



Model: "functional_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 299, 299, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 299, 299, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 149, 149, 64) 0           block1_conv2[0][0]               
_____________________________________________________________________________________